In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import ast

In [7]:
input_dir = "/content/drive/MyDrive/Colab/kaggle/nbme-score-clinical-patient-notes/input"
train  = pd.read_csv(os.path.join(input_dir, "train.csv"))
features = pd.read_csv(os.path.join(input_dir, "features.csv"))
pn = pd.read_csv(os.path.join(input_dir, "patient_notes.csv"))

In [9]:
pn_dict = {}
for idx, row in pn.iterrows():
    pn_dict[row['pn_num']] = row['pn_history']


new_annotation = []
for case_id in features['case_num'].unique():
    
    all_pn_id = set(pn[pn['case_num']==case_id]['pn_num'].tolist())
    
    for feature_id in features[features['case_num']==case_id]['feature_num'].unique():
        # get all the pn_num that have already been annotated
        annotated_pn = set(train[train['feature_num']==feature_id]['pn_num'].tolist())
        # get all the pn_num that have NOT been annotated
        pn_to_annotate = all_pn_id-annotated_pn
        
        # get all current annotations
        # we will use them to find more annotations
        annotations = train[train['feature_num']==feature_id]['annotation'].tolist()
        annotation_texts = set()
        for a in annotations:
            anns = eval(a)
            for at in anns:
                annotation_texts.add(at)
                
        # annotate       
        for pn_id in pn_to_annotate:
            new_annotation_pn, new_location_pn = [], []
            pn_text = pn_dict[pn_id]
            for at in annotation_texts:
                start = pn_text.find(at)
                if start>=0:
                    new_annotation_pn.append(at)
                    new_location_pn.append(f'{start} {start+len(at)}')
            if len(new_annotation_pn)>0:
                new_annotation.append((
                    f'{pn_id:04d}_{feature_id:03d}',
                    case_id,
                    pn_id,
                    feature_id,
                    new_annotation_pn,
                    new_location_pn
                ))

In [32]:
external = pd.DataFrame(new_annotation, columns=train.columns)
# external = external.merge(features, on=["feature_num", "case_num"], how="left")
# external = external.merge(pn, on=["pn_num", "case_num"], how="left")
# external.to_csv(os.path.join(input_dir, "external_exact_match_train.csv"), index=False)

In [34]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']


In [33]:
external_df = pd.read_csv(os.path.join(input_dir, "external_exact_match_train.csv"))
external_df

,id,case_num,pn_num,feature_num,annotation,location
0,0000_000,0,0,0,['father had MI'],['553 566']
1,0005_000,0,5,0,['father - MI'],['825 836']
2,0007_000,0,7,0,['Father MI'],['490 499']
3,0009_000,0,9,0,['father had MI'],['709 722']
4,0011_000,0,11,0,['Father had MI'],['684 697']
...,...,...,...,...,...,...
304147,95325_916,9,95325,916,"['subjective fever', 'fever']","['463 479', '474 479']"
304148,95327_916,9,95327,916,"['subjective fever', 'fever', 'subjective feve...","['302 318', '313 318', '302 319']"
304149,95328_916,9,95328,916,['FEVER'],['348 353']
304150,95332_916,9,95332,916,['Felt warm'],['493 502']
